In [ ]:
#Run if in Colab.
!git clone -b msnet https://<YourAccessTokenHERE>@github.com/jpradov/ipeo_project.git
from google.colab import drive
import os
import sys
#prepare a folder with your data
sys.path.append('/content/ipeo_project')
DATA_PATH = "/content/drive/MyDrive/ipeo_project/dataset/"
drive.mount('/content/drive')
!pip install kornia imagecodecs==2023.9.18

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingLR
from utils import data
import pandas as pd
from torch.utils.data import Subset, DataLoader
from torchvision import transforms
import config
from tqdm import tqdm
import kornia
import matplotlib.pyplot as plt
import numpy as np
from utils import evaluation

from models.msnet import MSNet
from torch.utils.data import DataLoader
from utils.data import create_dataloaders
from utils.train import train_epoch


In [ ]:
#Should be cuda:0 in colab and cpu in local.
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
#DATA_PATH = config.PATH_TO_DATA


In [ ]:
loader_params = {
    "batch_size" : 32,
    "batch_transforms" : True,
    "num_workers" : 2
}
train_loader, test_loader, val_loader = create_dataloaders(DATA_PATH, **loader_params)


In [ ]:
print(len(train_loader.torch_loader))

In [ ]:
%load_ext autoreload
%autoreload 2
max_epochs = 2

model = MSNet(num_classes=2).to(device)
learning_rate = 0.001
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()
loss_history, accuracy_history = train_epoch(model=model,device=device,train_loader=train_loader,optimizer=optimizer,epoch=1,criterion=criterion)


In [ ]:
%load_ext autoreload
%autoreload 2
test_loss, accuracy, iou, dice, precision, recall, f1 = evaluation.evaluate(model, device, val_loader, criterion)

print(accuracy, iou, dice, precision, recall, f1 )